In [ ]:
from pyqcu.gmg import *
from pyqcu import define, gauge, io, qcu, set
import cupy as cp
import numpy as np
from pyqcu.set import params, argv


class WilsonCase:
    def __init__(self, params=None, argv=None, min_size=4, max_levels=10, seed=12138):
        if params is None:
            self.params = set.params
            self.params[define._LAT_X_] = 4
            self.params[define._LAT_Y_] = 16
            self.params[define._LAT_Z_] = 16
            self.params[define._LAT_T_] = 4
            self.params[define._LAT_XYZT_] = self.params[define._LAT_X_] * \
                self.params[define._LAT_Y_]*self.params[define._LAT_Z_] * \
                self.params[define._LAT_T_]
        else:
            self.params = params
        if argv is None:
            self.argv = set.argv
        else:
            self.argv = argv
        self.min_size = min_size
        self.max_levels = max_levels
        self.seed = seed
        np.random.seed(seed)
        cp.random.seed(seed)

    def give_b(self, params):
        b = cp.ones(params[define._LAT_XYZT_]*define._LAT_SC_,
                    dtype=define.dtype(params[define._DATA_TYPE_]))
        b = io.fermion2psctzyx(b, params)
        return b

    def setup(self):
        grid_params = []
        current_nx, current_ny, current_nz = self.params[define._LAT_Y_], self.params[
            define._LAT_Z_], self.params[define._LAT_X_]*self.params[define._LAT_T_]*define._LAT_SC_
        print(
            f"current_nx: {current_nx}, current_ny: {current_ny}, current_nz: {current_nz}")
        while min(current_nx, current_ny) >= self.min_size and len(grid_params) < self.max_levels:
            grid_params.append((current_nx, current_ny, current_nz))
            print(
                f"  Level {len(grid_params)-1}: {current_nx}x{current_ny}x{current_nz}")
            current_nx = max(2, current_nx // 2)
            current_ny = max(2, current_ny // 2)
        set_ptrs = set.set_ptrs
        for i, (nx, ny, nz) in enumerate(grid_params):
            params = self.params
            argv = self.argv
            params[define._SET_INDEX_] = i
            params[define._SET_PLAN_] = define._SET_PLAN1_
            params[define._LAT_Y_] = nx
            params[define._LAT_Z_] = ny
            params[define._LAT_XYZT_] = params[define._LAT_X_] * \
                params[define._LAT_Y_]*params[define._LAT_Z_] * \
                params[define._LAT_T_]
            U = gauge.give_gauge(params=params)
            qcu.applyInitQcu(set_ptrs, params, argv)
            src = self.give_b(params)
            dest = cp.zeros_like(src)
            qcu.applyWilsonBistabCgQcu(dest, src,
                                       U, set_ptrs, params)
            print("Dest data:", dest.data)
            print("Dest shape:", dest.shape)


class WilsonOperator:
    def __init__(self, nx=32, ny=32, nz=12):
        self.params = params
        self.argv = argv

        self.nx = params[define._LAT_X_]
        self.ny = params[define._LAT_Y_]
        self.nz = params[define._LAT_Z_]
        self.nt = params[define._LAT_T_]
        self.n = self.nx * self.ny * self.nz * self.nt


    @@@@@@######QCU NOTES START######@@@@@@@
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (wilson:x=4,y=4,z=4,t=4;clover:x=8,y=8,z=8,t=8) that QCU support (when '#define _BLOCK_SIZE_ 

In [ ]:

# demo()


case = WilsonCase()

case.setup()


current_nx: 16, current_ny: 16, current_nz: 192
  Level 0: 16x16x192
  Level 1: 8x8x192
  Level 2: 4x4x192
